<a href="https://colab.research.google.com/github/ABBjaceng/AI_projekt_Jacob/blob/master/MoviesV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jacob engström
#### AI projekt - ABBindustrigymnasium 180S



# Installera bibliotek
```gpt2``` Använder jag för att träna medellen

```datetime``` Använder jag eftersom python inte har något imbyggd världsklocka använder jag datetime för att visa tiden när programet har tränat ett bästämt antal epoker

```files``` Använder jag för att kunda spara filer och modeller till Google Files

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install gTTS
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from gtts import gTTS 
import tensorflow as tf
import random
import tarfile
import requests
import os

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Installera GPT-2 model
Jag väljer `774M` eftersom den är tillräckligt bra modell för den applicationen jag vill. Det finns mindre modeller: `355M` och `124M`, desto mindre nummer desto mindre och mer osäker är modellen. Det finns även en större model `1558M`


In [0]:
gpt2.download_gpt2(model_name="774M")

# Mountar i Google Drive
Anslut till Google Drive genom att följa länken

In [2]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Träna på texter
Om du vill träna programet kör föjande kod, annars gå längre ner

In [0]:
!wget https://raw.githubusercontent.com/ABBjaceng/AI_projekt_Jacob/master/scripts.txt

`steps=350,` jag väljer att träna programet med 350 epoker. det kan verka lite lågt men jag tränar på 36000+ rader så jag kan inte ha för många epoker utan att den blir övertränad

In [0]:
sess = gpt2.start_tf_sess()
file_name = "scripts.txt"

gpt2.finetune(sess,
              dataset=file_name,
              model_name='774M',
              steps=350,
              restore_from='fresh',
              run_name='runscripts',
              print_every=10,
              sample_every=100,
              save_every=350
              )

gpt2.copy_checkpoint_to_gdrive(run_name='runscripts')

test med uppläsning

In [0]:
filepath="checkpoint_runscripts.tar"
googefileid= "1qx8pywi7HFirIHgduSU-qKNZqqQ4n8At"

def extract():
    """Copies the checkpoint folder from a mounted Google Drive."""
    with tarfile.open(filepath, 'r') as tar:
        tar.extractall()
    os.remove(filepath)
    print("File",filepath, "Removed!")

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

In [5]:
gpt2.copy_checkpoint_from_gdrive(run_name='runscripts')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='runscripts')

def createStory():
    temp= random.randint(7,9)/10
    print(temp)
    text= gpt2.generate(sess,
            run_name='runscripts',
            length=600,
            temperature=0.85,
            top_p=1,
            top_k=30,
            nsamples=1,
            batch_size=1,
            return_as_list=True
            )
            
    text= text[0]
    return text

def Story():
    text = createStory()
    Story = open(r"Story.txt","w+") 
    Story.write(text)
    print(text)

def MP3():
    fileHandler = open("Story.txt", "r")
    myText =fileHandler.read().replace("\n", " ")
    language = 'en'
    output = gTTS(text=myText, lang=language, slow=False)
    output.save("Story.mp3")
    fileHandler.close()
    os.system("start Story.mp3")

Story()
MP3()

ValueError: ignored

# Generera texter från model
Det finns några parametrar du kan ändra för att få andra resultat

`length` - antalet tecken som generas, max 1023

`temperature` - högre värde desto mer galet blir resultatet. Recomenderas mellan 0.7 till 1.0

`top_k` - begränsar antaltet gissningar, om resultatet är för galet sätt den till 40

In [0]:
text2 = gpt2.generate(sess, run_name='runscripts',
              length=600,
              temperature=0.85,
              top_p=1,
              top_k=30,
              nsamples=1,
              batch_size=1,
              return_as_list=False)

# LICENSE
MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.